In [ ]:
from lsst.daf.butler import Butler

In [ ]:
DATASTORE = '/work/datastore/'
COLLECTIONS = 'PFS/default'
INSTRUMENT = 'lsst.obs.pfs.PrimeFocusSpectrograph'
NUM_JOBS = 25

REDUCE_PIPELINE_PATH = '$DRP_STELLA_DIR/pipelines/reduceExposure.yaml'
QA_PIPELINE_PATH = f'/work/wtg/drp_qa_dev/pipelines/detectorMapQa.yaml'

OUTPUT_COLLECTION = 'wtg/PIPE2D-1528'
LOG_FILE = './pipetask.log'

In [ ]:
!defineCombination.py --update {DATASTORE} PFS wtg/run18_dm_inputs \
    113981 113982 113983 113984 113985 113986 113987 113988 113989 113990 \
    113991 113992 113993 113994 113995 114641 114642 114644 114645 114646 \
    114647 114648 114649 114650 114651 114652 114653 114654 114655 114656 \
    114657 114659 114660 114661 114986 114987 114988 114989 114990 114991 \
    114992 114993 114994 114995 114996 114997 114999 114500

## Run `reduceExposure` pipeline

This will generate the `lines` and adjusted `detectorMap` objects we use for the DetectorMap QA.

We use the `!` operator to call the `pipetask` command the same as we would in a shell.

In this example we use the combination defined above and use the data select option (`-d`) to limit
the processing to certain arms.

In [ ]:
!pipetask \
    --long-log \
    --no-log-tty \
    --log-level "pfs.drp.stella=INFO" \
    --log-file {LOG_FILE} \
    run \
    -b {DATASTORE} \
    -j {NUM_JOBS} \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p {REDUCE_PIPELINE_PATH} \
    -d "combination in ('wtg/run18_dm_origin_inputs') AND arm in ('b', 'r', 'n')"

## Run `detectorMapQa` pipeline

The `detectorMapQa` runs a number of separate QA related tasks.

We specify a few options to show the syntax but the default options should be
okay for most processing.

In [ ]:
use_sigma = False
spatial_range = 0.1
wavelength_range = 0.1

In [ ]:
!pipetask \
    --log-tty \
    --long-log \
    --log-level "pfs.detectorMapCombinedQa=INFO" \
    --log-file {LOG_FILE} \
    run \
    -b {DATASTORE} \
    -j {NUM_JOBS} \
    --instrument {INSTRUMENT} \
    -i {COLLECTIONS} \
    -o {OUTPUT_COLLECTION} \
    -p "{QA_PIPELINE_PATH}" \
    -d "combination = 'wtg/run18_dm_origin_inputs' AND arm in ('b', 'r', 'n')" \
    -c dmResiduals:useSigmaRange={use_sigma} \
    -c dmResiduals:spatialRange={spatial_range} \
    -c dmResiduals:wavelengthRange={wavelength_range} \
    --fail-fast

## Get stored objects from the butler

We use the `OUTPUT_COLLECTION` to look up the objects stored by the tasks.

In [ ]:
butler = Butler(DATASTORE, collections=OUTPUT_COLLECTION)


The QA task for individual exposures produces a plot (saved as a `.png` image)
and a `pandas.DataFrame` that has two rows of statistics, one for the `USED`
points and one for the `RESERVED` points.

The `USED` points come from the adjustment itself and the `RESERVED` points are
 what we want to do the QA checks on.

The statistics are stored under the `dmQaResidualStats` and the plot is `dmQaResidualPlot`. These both require the `(arm, spectrograph, exposure)` dimensions, which we define here as a traditional DataId.

In [ ]:
data_id = dict(arm='r', spectrograph=2, exposure=113595)

In [ ]:
# Show the RESERVED and USED stats for the given detector exposure.
butler.get('dmQaResidualStats', data_id).T

In [ ]:
Image(filename=butler.getURI('dmQaResidualPlot', data_id).unquoted_path)

In [ ]:
butler.get('dmQaDetectorStats')

In [ ]:
butler.getURI('dmQaCombinedResidualPlot')

In [ ]:
!cp {butler.getURI('dmQaCombinedResidualPlot').unquoted_path} .

In [ ]:
butler.get('dmQaResidualStats', data_id).T

In [ ]:
Image(filename=butler.getURI('dmQaResidualPlot', data_id).unquoted_path)